In [1]:
import requests
import json
import StaticAnalysisGraphBuilder
import AI4MLTagReader
import rdflib
import numpy as np
import os
import sys

"""
  This class opens up a pandas dataframe of code from github, and calls WALA's apis to get control flow and data flow
  for each class.  The class is broken down into a set of entry points that correspond to each function in the dataset.
  Analysis starts at each entry point, and returns a graph of control flow and data flow edges.  These are 
  converted into an RDF graph and dumped in turtle.
"""
classes_to_superclasses = AI4MLTagReader.get_class_hierarchy()

url = 'http://localhost:4567/analyze_code'


def main(inputdir, graphdir, limit=-1):
    fails = 0
    success = 0

    for i, f in enumerate(os.listdir(inputdir)):
        if limit >= 0 and i > limit:
            break
        if not f.startswith('sample'):
            continue
        with open(os.path.join(inputdir, f)) as sample_file:
            source = sample_file.read()

        graph_tuple = handle_call_to_analysis(source, f)
        if graph_tuple:
            single_g = rdflib.Graph()
            StaticAnalysisGraphBuilder.addToGraph(single_g, graph_tuple)
            fn = os.path.join(graphdir, f + '.ttl')
            print(fn)
            with open(fn, 'wb') as out:
                out.write(single_g.serialize(format='turtle'))
            success += 1
        else:
            fails += 1

    print("num successes:" + str(success))
    print('num failures' + str(fails))

main('../kaggle/python_files','../kaggle_rdf',10)

NameError: name 'handle_call_to_analysis' is not defined